# Import Dependecies

In [12]:
import pandas as pd
import numpy as np
import math
import string
import PyPDF2
import nltk
import textract
from nltk import sent_tokenize
from nltk import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
from nltk import tokenize

In [ ]:
#!pip install 'PyPDF2<3.0'
#!pip install textract
#!pip install -U nltk
#!python -m nltk.downloader all

The system cannot find the file specified.


### 1. Load PDF

In [15]:
#pdf = open('Commercial_Security_System_Market.pdf','rb')
# output = textract.process('Commercial_Security_System_Market.pdf').decode('utf-8')
import fitz  # PyMuPDF

with fitz.open('Commercial_Security_System_Market.pdf') as pdf:
    output = ''.join([pdf[page_num].get_text() for page_num in range(pdf.page_count)])

print(output)


 
 
2 
COMMERCIAL SECURITY SYSTEM MARKET WITH COVID-19 IMPACT ANALYSIS - GLOBAL FORECAST TO 2026 
DECEMBER 2020 © MarketsandMarkets™ 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
MarketsandMarkets™ is the world’s largest revenue impact company, serving over 7500 customers. 80% of 
top 2000 companies globally rely on us for identifying new high growth and niche revenue opportunities. 
 
In the face of constant technology innovation and market disruption, we help organizations plan and 
operationalize their future revenue mix decisions by identifying over 30,000 high growth opportunities 
ranging from $1B to $500B across 90+ industry trends and markets. Organizations choose 
MarketsandMarkets™ to stay ahead of the curve and accelerate their revenue decisions and 
implementations by 6 – 12 months, giving them a unique, first-mover advantage.  
 
Our revenue impact methodology provides quantified and actionable insights on converged, granular and 
connected market eco-sy

In [16]:
output

' \n \n2 \nCOMMERCIAL SECURITY SYSTEM MARKET WITH COVID-19 IMPACT ANALYSIS - GLOBAL FORECAST TO 2026 \nDECEMBER 2020 © MarketsandMarkets™ \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \nMarketsandMarkets™ is the world’s largest revenue impact company, serving over 7500 customers. 80% of \ntop 2000 companies globally rely on us for identifying new high growth and niche revenue opportunities. \n \nIn the face of constant technology innovation and market disruption, we help organizations plan and \noperationalize their future revenue mix decisions by identifying over 30,000 high growth opportunities \nranging from $1B to $500B across 90+ industry trends and markets. Organizations choose \nMarketsandMarkets™ to stay ahead of the curve and accelerate their revenue decisions and \nimplementations by 6 – 12 months, giving them a unique, first-mover advantage.  \n \nOur revenue impact methodology provides quantified and actionable insights on c

In [17]:
def text_extraction(pdf):
    pdfReader = PyPDF2.PdfFileReader(pdf)
    count = pdfReader.numPages
    output = ''

    for i in range(count):
        page = pdfReader.getPage(i)
        output += (page.extractText()).replace('\n','')
    return output

In [24]:
# read text and tokenize
#output = text_extraction(pdf)
import nltk
nltk.download('punkt')
sentences = tokenize.sent_tokenize(output)


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\HomePC\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [25]:
output

'Your text extracted from PDF goes here'

In [26]:
sentences

['Your text extracted from PDF goes here']

In [27]:
len(sentences)

1

### 2. Clean text (words & sentences)

In [28]:
def clean_sentences(sentences):
    stopWords = set(stopwords.words("english"))
    lemmatizer = WordNetLemmatizer()
    clean_document_sent = []

    for sent in sentences:
        words = word_tokenize(sent)
        clean_sentence = []
        for word in words:
            word = word.lower()
            word = lemmatizer.lemmatize(word)
            # remove punctuation from each word
            table = str.maketrans('','',string.punctuation)
            word = word.translate(table)
            if word.isalpha() == False:
                continue
            if word in stopWords:
                continue
            if (len(word)<4):
                continue
            clean_sentence.append(word)
        clean_document_sent.append(clean_sentence)
    return clean_document_sent

In [29]:
doc_sent_clean = clean_sentences(sentences)
len(doc_sent_clean)

1

In [30]:
doc_sent_clean

[['text', 'extracted']]

In [31]:
doc_sent_clean = [i for i in doc_sent_clean if len(i)>3]
len(doc_sent_clean)

0

In [32]:
doc_sent_clean

[]

In [33]:
doc_word_clean =  [item for sublist in doc_sent_clean for item in sublist]

In [34]:
doc_word_clean 

[]

In [35]:
len(doc_word_clean)

0

### 3. Tf-Idf and top keywords

In [36]:
# tf score function
def tf_score(doc_word_clean):
    tf_score = {}
    for each_word in doc_word_clean:
        if each_word in tf_score:
            tf_score[each_word] += 1
        else:
            tf_score[each_word] = 1

    # Dividing by total_word_length for each dictionary element
    total_word_length = len(doc_word_clean)
    tf_score.update((x, y/int(total_word_length)) for x, y in tf_score.items())
    return tf_score

In [37]:
# save tf scores 
tf_score = tf_score(doc_word_clean)

In [38]:
tf_score

{}

In [39]:
def check_sent(word, sentences): 
    final = [all([w in x for w in word]) for x in sentences] 
    sent_len = [sentences[i] for i in range(0, len(final)) if final[i]]
    return int(len(sent_len))

In [40]:
# transform sentences for idf method
transform_sentences = []
for i in doc_sent_clean:
    transform_sentences.append(' '.join(i))

In [41]:
transform_sentences

[]

In [42]:
def idf_calc_score(doc_word_clean,transform_sentences):
    idf_score = {}
    for each_word in doc_word_clean:
        each_word = each_word.replace('.','')
        if each_word in idf_score:
            idf_score[each_word] = check_sent(each_word, transform_sentences)
        else:
            idf_score[each_word] = 1
    
    # Performing a log and divide
    total_sent_len = len(transform_sentences)
    idf_score.update((x, math.log(int(total_sent_len)/y)) for x, y in idf_score.items())
    return idf_score

In [43]:
# save idf scores
idf_score = idf_calc_score(doc_word_clean,transform_sentences)

In [44]:
idf_score

{}

In [45]:
# save tf-idf scores
tf_idf_score = {key: tf_score[key] * idf_score.get(key, 0) for key in tf_score.keys()}

In [46]:
tf_idf_score

{}

In [47]:
# select top key words
from operator import itemgetter
def get_top_n(dict_elem, n):
    result = dict(sorted(dict_elem.items(), key = itemgetter(1), reverse = True)[:n]) 
    return result

In [48]:
key_words_new = get_top_n(tf_idf_score, 10)

In [ ]:
# remove words like 'table', 'page' or 'figure'

In [49]:
key_words_new

{}

### 4. Generate PDF vector based on key words

In [53]:
from sentence_transformers import SentenceTransformer
#paraphrase-mpnet-base-v2 corpus of data used to train the model
model_bert = SentenceTransformer('paraphrase-mpnet-base-v2')

ModuleNotFoundError: No module named 'sentence_transformers'

In [51]:
def vector_pdf_embedding(key_words_new):
    # calculate weights for key words
    key_words_weights = np.array(list(key_words_new.values()))/np.sum(np.array(list(key_words_new.values())))
    print(key_words_weights)
    # generate embeddings for each key word from top n list
    embeddings_key_words = model_bert.encode(list(key_words_new.keys()))
    # generate a embedding vector for pdf based on top key words
    vector_pdf = np.dot(key_words_weights,embeddings_key_words)
    return vector_pdf

In [ ]:
embeddings_key_words = model_bert.encode(list(key_words_new.keys()))

In [ ]:
key_words_new

In [ ]:
embeddings_key_words.shape

In [ ]:
embeddings_key_words

In [ ]:
key_words_new

In [ ]:
embeddings_key_words[0]

In [ ]:
# save pdf embeddings
vector_pdf = vector_pdf_embedding(key_words_new)

In [ ]:
len(vector_pdf)

In [ ]:
vector_pdf

In [ ]:
# read industries embeddings 
industry = pd.read_csv('industry_improved_bert_embedding.csv')

In [ ]:
industry

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

In [ ]:
similarity_df = pd.DataFrame(((cosine_similarity(industry.iloc[:,1:],industry.iloc[:,1:])+1)/2),index = industry.iloc[:,0],columns =industry.iloc[:,0])

In [ ]:
similarity_df['industry_name'] = similarity_df.index

In [ ]:
similarity_df

In [ ]:
# Importing Modules
from scipy.cluster.hierarchy import linkage, dendrogram
import matplotlib.pyplot as plt
plt.figure(figsize=(16,8))
import pandas as pd

# Reading the DataFrame
df = similarity_df

# Remove the grain species from the DataFrame, save for later
varieties = list(df.pop('industry_name'))

# Extract the measurements as a NumPy array
samples = df.values


mergings = linkage(samples, method='complete')


dendrogram(mergings,
           labels=varieties,
           leaf_rotation=90,
           leaf_font_size=6,
           )

plt.show()

In [ ]:
from scipy.cluster.hierarchy import fcluster
fl = fcluster(mergings,150,criterion='maxclust')

In [ ]:
similarity_df['cluster'] = fl

In [ ]:
cols =similarity_df.columns.tolist()
cols = cols[-1:] + cols[:-1]
similarity_df = similarity_df[cols]

In [ ]:
similarity_df = similarity_df.sort_values(by = 'cluster',ascending = True)

In [ ]:
similarity_df.to_csv('clusters_of_similar_industries.csv')

In [ ]:
# generate sorted list of similar industries
df = pd.DataFrame(vector_pdf)
similarity_df = pd.DataFrame(((cosine_similarity(df.T,industry.iloc[:,1:])+1)/2),index = ['pdf'],columns =industry.iloc[:,0])
sorted_similarity = (similarity_df.sort_values(axis=1,by='pdf',ascending=False)).transpose()
sorted_similarity.head(30)

In [ ]:
sorted_similarity.to_csv('sorted_similarity_industries_list_Commercial_Security_System_Market.csv')

### 5. Code for generating embeddings for industries

In [ ]:
#industry = pd.read_csv('industry_bert_embedding.csv')

In [ ]:
bert_embedding = []
for i in industry.industry:
    bert_embedding.append(model_bert.encode(i.replace('_',' ')))
df = pd.DataFrame(bert_embedding, index =industry.industry,columns = range(0,768))

In [ ]:
df.to_csv('industry_improved_bert_embedding.csv')